In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [20]:
from src.inference import get_feature_store

In [21]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-02-27 16:26:53,863 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-27 16:26:53,926 INFO: Initializing external client
2025-02-27 16:26:53,927 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-27 16:26:54,988 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648
Fetching data from 2025-01-29 21:26:53.854375+00:00 to 2025-02-27 20:26:53.854375+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.97s) 


In [22]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-01-29 22:00:00+00:00,2,0
1,2025-01-29 23:00:00+00:00,2,0
2,2025-01-30 00:00:00+00:00,2,0
3,2025-01-30 01:00:00+00:00,2,0
4,2025-01-30 02:00:00+00:00,2,0
...,...,...,...
174919,2025-02-26 23:00:00+00:00,263,44
174920,2025-02-27 00:00:00+00:00,263,28
174921,2025-02-27 01:00:00+00:00,263,17
174922,2025-02-27 02:00:00+00:00,263,2


In [23]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174924 entries, 0 to 181115
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174924 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174924 non-null  int32                  
 2   rides               174924 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [24]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [25]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174924 entries, 0 to 181115
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174924 non-null  datetime64[us]
 1   pickup_location_id  174924 non-null  int32         
 2   rides               174924 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [26]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [27]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,84,2025-02-16 03:00:00
1,0,2,0,0,0,1,0,0,1,1,...,0,1,1,1,0,0,0,0,189,2025-02-26 16:00:00
2,1,0,0,1,0,0,1,1,0,1,...,1,0,2,2,1,0,0,0,197,2025-02-26 17:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,156,2025-02-02 03:00:00
4,0,0,2,1,0,0,0,1,2,1,...,1,0,3,1,0,0,0,0,247,2025-02-27 03:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,58,2025-02-27 01:00:00
254,0,0,1,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,183,2025-02-27 02:00:00
255,52,18,23,120,31,76,107,36,105,247,...,75,381,51,0,27,47,39,91,79,2025-02-26 18:00:00
256,154,253,267,108,355,87,289,87,154,367,...,390,58,332,209,327,1,308,61,132,2025-02-26 16:00:00


In [28]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-02-27 16:27:23,886 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-27 16:27:23,893 INFO: Initializing external client
2025-02-27 16:27:23,894 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-27 16:27:24,633 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648
Fetching data from 2025-01-29 21:27:23.886578+00:00 to 2025-02-27 20:27:23.886578+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.53s) 


In [29]:
current_date

Timestamp('2025-02-27 21:27:23.886578+0000', tz='Etc/UTC')

In [30]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-02-26 22:00:00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-02-26 22:00:00
2,0,0,5,5,2,0,0,0,2,7,...,2,3,3,3,5,1,10,1,4,2025-02-26 22:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,2025-02-26 22:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-02-26 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,0,0,0,0,0,0,0,1,4,1,...,0,1,0,1,0,0,0,0,259,2025-02-26 22:00:00
254,0,0,0,0,0,0,1,2,0,4,...,0,1,0,0,1,3,1,1,260,2025-02-26 22:00:00
255,0,0,3,0,1,1,0,2,2,8,...,57,29,45,62,64,49,27,34,261,2025-02-26 22:00:00
256,0,0,3,2,0,2,7,16,66,164,...,89,111,83,104,99,52,34,21,262,2025-02-26 22:00:00


In [31]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-02-27 16:27:43,073 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-27 16:27:43,084 INFO: Initializing external client
2025-02-27 16:27:43,084 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-27 16:27:43,889 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648

In [32]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [33]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,4.0
3,5,0.0
4,6,0.0
...,...,...
253,259,0.0
254,260,0.0
255,261,32.0
256,262,12.0


In [34]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([230, 161, 163, 142, 132, 234, 249, 237,  48, 162], dtype=int32)